# 作业

更换TokenEmbedding预训练模型，使用VisualDL查看相应的TokenEmbedding可视化效果，并尝试更换后的TokenEmbedding计算句对语义相似度。
本作业详细步骤，可参考[Day01作业教程](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/education/day01.md)，记得star PaddleNLP，收藏起来，随时跟进最新功能噢。

**作业结果提交**：
1. 截图提交可视化结果（图片注明作业可视化结果）。
2. 通篇执行每段代码，并保留执行结果。

# PaddleNLP词向量应用展示

6.7日NLP直播打卡课开始啦

**[直播链接请戳这里，每晚20:00-21:30👈](http://live.bilibili.com/21689802)**

**[课程地址请戳这里👈](https://aistudio.baidu.com/aistudio/course/introduce/24177)**

欢迎来课程**QQ群**（群号:618354318）交流吧~~


词向量（Word embedding），即把词语表示成实数向量。“好”的词向量能体现词语直接的相近关系。词向量已经被证明可以提高NLP任务的性能，例如语法分析和情感分析。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/54878855b1df42f9ab50b280d76906b1e0175f280b0f4a2193a542c72634a9bf" width="60%" height="50%"> <br />
</p>
<br><center>图1：词向量示意图</center></br>

PaddleNLP已预置多个公开的预训练Embedding，您可以通过使用`paddlenlp.embeddings.TokenEmbedding`接口加载预训练Embedding，从而提升训练效果。本篇教程将依次介绍`paddlenlp.embeddings.TokenEmbedding`的初始化和文本表示效果，并通过文本分类训练的例子展示其对训练提升的效果。

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 430kB 16kB/s eta 0:00:012
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [2]:
import paddlenlp


paddlenlp.embeddings.list_embedding_name()

['w2v.baidu_encyclopedia.target.word-word.dim300',
 'w2v.baidu_encyclopedia.target.word-character.char1-1.dim300',
 'w2v.baidu_encyclopedia.target.word-character.char1-2.dim300',
 'w2v.baidu_encyclopedia.target.word-character.char1-4.dim300',
 'w2v.baidu_encyclopedia.target.word-ngram.1-2.dim300',
 'w2v.baidu_encyclopedia.target.word-ngram.1-3.dim300',
 'w2v.baidu_encyclopedia.target.word-ngram.2-2.dim300',
 'w2v.baidu_encyclopedia.target.word-wordLR.dim300',
 'w2v.baidu_encyclopedia.target.word-wordPosition.dim300',
 'w2v.baidu_encyclopedia.target.bigram-char.dim300',
 'w2v.baidu_encyclopedia.context.word-word.dim300',
 'w2v.baidu_encyclopedia.context.word-character.char1-1.dim300',
 'w2v.baidu_encyclopedia.context.word-character.char1-2.dim300',
 'w2v.baidu_encyclopedia.context.word-character.char1-4.dim300',
 'w2v.baidu_encyclopedia.context.word-ngram.1-2.dim300',
 'w2v.baidu_encyclopedia.context.word-ngram.1-3.dim300',
 'w2v.baidu_encyclopedia.context.word-ngram.2-2.dim300',
 'w2v.

## 加载TokenEmbedding

`TokenEmbedding()`参数
- `embedding_name`
将模型名称以参数形式传入TokenEmbedding，加载对应的模型。默认为`w2v.baidu_encyclopedia.target.word-word.dim300`的词向量。
- `unknown_token`
未知token的表示，默认为[UNK]。
- `unknown_token_vector`
未知token的向量表示，默认生成和embedding维数一致，数值均值为0的正态分布向量。
- `extended_vocab_path`
扩展词汇列表文件路径，词表格式为一行一个词。如引入扩展词汇列表，trainable=True。
- `trainable`
Embedding层是否可被训练。True表示Embedding可以更新参数，False为不可更新。默认为True。

In [3]:
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
# 需要更换所选的词向量
token_embedding = TokenEmbedding(embedding_name="w2v.zhihu.target.word-bigram.dim300")

# 查看token_embedding详情
print(token_embedding)

100%|██████████| 284195/284195 [00:07<00:00, 39555.51it/s]
[2021-06-08 22:52:41,628] [    INFO] - Loading token embedding...
[2021-06-08 22:52:45,859] [    INFO] - Finish loading embedding vector.
[2021-06-08 22:52:45,862] [    INFO] - Token Embedding info:             
Unknown index: 259883             
Unknown token: [UNK]             
Padding index: 259884             
Padding token: [PAD]             
Shape :[259885, 300]


Object   type: TokenEmbedding(259885, 300, padding_idx=259884, sparse=False)             
Unknown index: 259883             
Unknown token: [UNK]             
Padding index: 259884             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[259885, 300], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-0.80178398, -0.16534001,  0.03050800, ...,  0.10652500,  0.55343598,  0.43665001],
        [-0.65174699,  0.53596997,  0.34027100, ...,  0.80539900,  0.10459300,  0.19369400],
        [-0.41232100,  0.22826099,  0.20711400, ...,  0.80877697,  0.05675100,  0.45237401],
        ...,
        [ 0.00697300, -0.00230300,  0.00643700, ...,  0.00670700,  0.00719100, -0.00570000],
        [ 0.00346854,  0.01151508,  0.00216891, ..., -0.03975425,  0.01145951, -0.01988915],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  0.        ]])


### 认识一下Embedding
**`TokenEmbedding.search()`**
获得指定词汇的词向量。

In [4]:
test_token_embedding = token_embedding.search("知乎")
print(test_token_embedding)

[[ 3.46853933e-03  1.15150781e-02  2.16890872e-03 -2.48674694e-02
  -3.63955684e-02 -7.80402543e-03  3.86176892e-02 -1.34944692e-02
   2.83305291e-02 -2.92975381e-02 -2.55667698e-03 -5.63069899e-03
  -2.71602720e-02 -5.97354677e-03  2.29197508e-03  2.51236688e-02
   9.09854751e-03  3.68179418e-02 -2.32525356e-02  2.22933050e-02
  -2.02333536e-02  2.47712224e-03 -1.94173828e-02  3.71231735e-02
  -9.92194284e-04 -8.11171625e-03 -2.16184966e-02  4.03246954e-02
   1.06684426e-02 -1.86423343e-02 -7.68032391e-04  1.96365379e-02
   2.45762728e-02  4.26464947e-03  1.19809601e-02 -5.55391249e-04
   1.23041328e-02  1.94248483e-02 -5.40137384e-03 -1.07589550e-02
   2.38640141e-02 -3.33913439e-03 -1.38543332e-02  5.84536511e-03
   1.40434066e-02 -1.02990707e-02 -1.55742690e-02 -1.01696514e-02
   2.94366200e-02 -3.59412059e-02 -1.42303286e-02  5.17162532e-02
   1.80803295e-02  6.58793049e-03  2.55285692e-03  1.97233558e-02
   5.58885746e-03 -2.07183119e-02 -4.28438559e-03 -2.33794912e-03
  -1.37372

**`TokenEmbedding.cosine_sim()`**
计算词向量间余弦相似度，语义相近的词语余弦相似度更高，说明预训练好的词向量空间有很好的语义表示能力。

In [5]:
score1 = token_embedding.cosine_sim("公知", "教授")
score2 = token_embedding.cosine_sim("科技", "美国")
print('score1:', score1)
print('score2:', score2)

score1: 0.21395943
score2: 0.16048971


In [9]:
score3 = token_embedding.cosine_sim('清华大学', '臭豆腐')
score4 = token_embedding.cosine_sim('我去北京天安门找你玩', '我去国家大剧院听音乐会')
print('score3: ', score3)
print('score4: ', score4)

score3:  0.15838934
score4:  1.0000001


### 词向量映射到低维空间

使用深度学习可视化工具[VisualDL](https://github.com/PaddlePaddle/VisualDL)的[High Dimensional](https://github.com/PaddlePaddle/VisualDL/blob/develop/docs/components/README_CN.md#High-Dimensional--%E6%95%B0%E6%8D%AE%E9%99%8D%E7%BB%B4%E7%BB%84%E4%BB%B6)组件可以对embedding结果进行可视化展示，便于对其直观分析，步骤如下：

1. 升级 VisualDL 最新版本。

`pip install --upgrade visualdl`

2. 创建LogWriter并将记录词向量。

3. 点击左侧面板中的可视化tab，选择‘token_hidi’作为文件并启动VisualDL可视化

In [10]:
!pip install --upgrade visualdl

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 2.7MB 13.8MB/s eta 0:00:01
  Found existing installation: visualdl 2.1.1
    Uninstalling visualdl-2.1.1:
      Successfully uninstalled visualdl-2.1.1


In [11]:
# 获取词表中前1000个单词
labels = token_embedding.vocab.to_tokens(list(range(0, 1000)))
# 取出这1000个单词对应的Embedding
test_token_embedding = token_embedding.search(labels)

# 引入VisualDL的LogWriter记录日志
from visualdl import LogWriter

with LogWriter(logdir='./token_hidi') as writer:
    writer.add_embeddings(tag='test', mat=[i for i in test_token_embedding], metadata=labels)

#### 启动VisualDL查看词向量降维效果
启动步骤：
- 1、切换到「可视化」指定可视化日志
- 2、日志文件选择 'token_hidi'
- 3、点击「启动VisualDL」后点击「打开VisualDL」，选择「高维数据映射」，即可查看词表中前1000词UMAP方法下映射到三维空间的可视化结果:

![](https://ai-studio-static-online.cdn.bcebos.com/3b05b744e9d9410bb59864064af857245a883c0428d441bd8dd38bd9662e0bb6)


可以看出，语义相近的词在词向量空间中聚集(如数字、章节等)，说明预训练好的词向量有很好的文本表示能力。

使用VisualDL除可视化embedding结果外，还可以对标量、图片、音频等进行可视化，有效提升训练调参效率。关于VisualDL更多功能和详细介绍，可参考[VisualDL使用文档](https://github.com/PaddlePaddle/VisualDL/tree/develop/docs)。

## 基于TokenEmbedding衡量句子语义相似度

在许多实际应用场景（如文档检索系统）中， 需要衡量两个句子的语义相似程度。此时我们可以使用词袋模型（Bag of Words，简称BoW）计算句子的语义向量。

**首先**，将两个句子分别进行切词，并在TokenEmbedding中查找相应的单词词向量（word embdding）。

**然后**，根据词袋模型，将句子的word embedding叠加作为句子向量（sentence embedding）。

**最后**，计算两个句子向量的余弦相似度。

### 基于TokenEmbedding的词袋模型


使用`BoWEncoder`搭建一个BoW模型用于计算句子语义。

* `paddlenlp.TokenEmbedding`组建word-embedding层
* `paddlenlp.seq2vec.BoWEncoder`组建句子建模层


In [12]:
import paddle
import paddle.nn as nn
import paddlenlp


class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def get_cos_sim(self, text_a, text_b):
        text_a_embedding = self.forward(text_a)
        text_b_embedding = self.forward(text_b)
        cos_sim = self.cos_sim_func(text_a_embedding, text_b_embedding)
        return cos_sim

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

### 构造Tokenizer
使用TokenEmbedding词表构造Tokenizer。

In [13]:
from data import Tokenizer
tokenizer = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)

### 相似句对数据读取

以提供的样例数据text_pair.txt为例，该数据文件每行包含两个句子。


In [14]:
text_pairs = {}
with open("text_pair.txt", "r", encoding="utf8") as f:
    for line in f:
        text_a, text_b = line.strip().split("\t")
        if text_a not in text_pairs:
            text_pairs[text_a] = []
        text_pairs[text_a].append(text_b)

### 查看相似语句相关度

In [15]:
for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        print("text_a: {}".format(text_a))
        print("text_b: {}".format(text_b))
        print("cosine_sim: {}".format(model.get_cos_sim(text_a_ids, text_b_ids).numpy()[0]))
        print()

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 多项式矩阵的左共轭积及其应用
cosine_sim: 0.8554719090461731

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 退化阻尼对高维可压缩欧拉方程组经典解的影响
cosine_sim: 0.7698296308517456

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Burgers方程基于特征正交分解方法的数值解法研究
cosine_sim: 0.7925785183906555

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 有界对称域上解析函数空间的若干性质
cosine_sim: 0.7461780309677124

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于卷积神经网络的图像复杂度研究与应用
cosine_sim: 0.6947051286697388

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Cartesian发射机中线性功率放大器的研究
cosine_sim: 0.7624534964561462

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: CFRP加固WF型梁侧扭屈曲的几何非线性有限元分析
cosine_sim: 0.7951616048812866

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于线性CCD自适应成像的光刻机平台调平方法研究
cosine_sim: 0.7154415249824524

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于变分贝叶斯理论的图像复原方法研究
cosine_sim: 0.6473516225814819

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 网格资源分配中混合并行蚁群算法方式研究
cosine

### 使用VisualDL查看句子向量

In [16]:
# 引入VisualDL的LogWriter记录日志
import numpy as np
from visualdl import LogWriter    
# 获取句子以及其对应的向量
label_list = []
embedding_list = []

for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    embedding_list.append(model(text_a_ids).flatten().numpy())
    label_list.append(text_a)

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        embedding_list.append(model(text_b_ids).flatten().numpy())
        label_list.append(text_b)


with LogWriter(logdir='./sentence_hidi') as writer:
    writer.add_embeddings(tag='test', mat=embedding_list, metadata=label_list)

### 启动VisualDL观察句子向量降维效果

步骤如上述观察词向量降维效果一模一样。

![](https://ai-studio-static-online.cdn.bcebos.com/1918523f565744fe989d010cec2be69e961655d217ea4196a81763ec698059de)



可以看出，语义相近的句子在句子向量空间中聚集(如有关课堂的句子、有关化学描述句子等)。

# PaddleNLP更多预训练词向量
PaddleNLP提供61种可直接加载的预训练词向量，训练自多领域中英文语料、如百度百科、新闻语料、微博等，覆盖多种经典词向量模型（word2vec、glove、fastText）、涵盖不同维度、不同语料库大小，详见[PaddleNLP Embedding API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/model_zoo/embeddings.md)。

# 预训练词向量辅助分类任务

想学习词向量更多应用，来试试预训练词向量对分类模型的改善效果吧，[这里](https://aistudio.baidu.com/aistudio/projectdetail/1283423) 试试把`paddle.nn.Embedding`换成刚刚学到的预训练词向量吧。

# 加入课程交流群，一起学习吧

现在就加入课程群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >



**[直播链接请戳这里，每晚20:00-21:30👈](http://live.bilibili.com/21689802)**

**[还没有报名课程？赶紧戳这里，课程、作业安排统统在课程区哦👉🏻](https://aistudio.baidu.com/aistudio/course/introduce/24177)**